In [1]:
!nvidia-smi

Wed Feb 21 09:17:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.01                 Driver Version: 546.01       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce 930MX         WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A    0C    P5              N/A / 200W |    143MiB /  2048MiB |     38%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

d:\ML\objectDetection


In [3]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196  Python-3.11.5 torch-2.2.0+cpu CPU (Intel Core(TM) i5-6200U 2.30GHz)
Setup complete  (4 CPUs, 11.9 GB RAM, 709.3/931.5 GB disk)


In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="P7lpDjDnnF6of50XsAdz")
project = rf.workspace("apd-detection").project("apd-detection-vrpms")
dataset = project.version(1).download("yolov8")


loading Roboflow workspace...
loading Roboflow project...


In [31]:
# Train Dataset
%cd {HOME}

!yolo task=detect mode=train model=yolov8n.pt data={dataset.location}/data.yaml epochs=25 imgsz=128

d:\ML\objectDetection


^C


In [6]:
# Validate Model
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train2/weights/best.pt data={dataset.location}/data.yaml

d:\ML\objectDetection


Ultralytics YOLOv8.0.196  Python-3.11.5 torch-2.2.0+cpu CPU (Intel Core(TM) i5-6200U 2.30GHz)
Model summary (fused): 168 layers, 3007013 parameters, 0 gradients, 8.1 GFLOPs

val: Scanning D:\ML\objectDetection\apd-detection-1\valid\labels.cache... 47 images, 0 backgrounds, 0 corrupt: 100%|██████████| 47/47 [00:00<?, ?it/s]
val: Scanning D:\ML\objectDetection\apd-detection-1\valid\labels.cache... 47 images, 0 backgrounds, 0 corrupt: 100%|██████████| 47/47 [00:00<?, ?it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/3 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  33%|███▎      | 1/3 [00:00<00:01,  1.68it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  67%|██████▋   | 2/3 [00:01<00:00,  1.35it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00

In [8]:
# Predict
%cd {HOME}

!yolo task=detect mode=predict model={HOME}/runs/detect/train2/weights/best.pt conf=0.25 source={HOME}/sample.jpg

d:\ML\objectDetection


Ultralytics YOLOv8.0.196  Python-3.11.5 torch-2.2.0+cpu CPU (Intel Core(TM) i5-6200U 2.30GHz)
Model summary (fused): 168 layers, 3007013 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 d:\ML\objectDetection\sample.jpg: 96x128 (no detections), 46.8ms
Speed: 1.0ms preprocess, 46.8ms inference, 2.1ms postprocess per image at shape (1, 3, 96, 128)
Results saved to runs\detect\predict4
 Learn more at https://docs.ultralytics.com/modes/predict


In [22]:
# Sample using model Pre-trained using Roboflow
rf = Roboflow(api_key="P7lpDjDnnF6of50XsAdz")
project = rf.workspace().project("apd-detection-vrpms")
model = project.version(1).model

# infer on a local image
print(model.predict("img/sample.jpg", confidence=40, overlap=30).json())

# visualize your prediction
# model.predict("your_image.jpg", confidence=40, overlap=30).save("prediction.jpg")

# infer on an image hosted elsewhere
# print(model.predict("URL_OF_YOUR_IMAGE", hosted=True, confidence=40, overlap=30).json())

loading Roboflow workspace...
loading Roboflow project...
{'predictions': [{'x': 1716, 'y': 615, 'width': 36, 'height': 33, 'confidence': 0.8425694704055786, 'class': 'helm', 'class_id': 3, 'detection_id': '0dbc4b65-9fe6-4904-848d-f5283dec3656', 'image_path': 'sample.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 1707, 'y': 800, 'width': 30, 'height': 35, 'confidence': 0.7858035564422607, 'class': 'boots', 'class_id': 0, 'detection_id': '38752a4b-cbba-48fb-ab65-45c825bf45c5', 'image_path': 'sample.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 1747, 'y': 714, 'width': 21, 'height': 32, 'confidence': 0.6646450757980347, 'class': 'noglove', 'class_id': 4, 'detection_id': 'af3a0ff1-08fb-4caa-8be2-1aad2df3215e', 'image_path': 'sample.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 1650, 'y': 239, 'width': 18, 'height': 16, 'confidence': 0.6300569176673889, 'class': 'helm', 'class_id': 3, 'detection_id': 'fc981a2f-8040-4d54-9ec4-0244fd2691b4', 'image_path': 'sample

In [27]:
# visualize your prediction
model.predict("img/sample2.jpg", confidence=40, overlap=30).save("prediction2.jpg")

In [9]:
rf = Roboflow(api_key="P7lpDjDnnF6of50XsAdz")
project = rf.workspace().project("apd-detection-vrpms")
model = project.version(1).model

loading Roboflow workspace...
loading Roboflow project...
